In [20]:
import random
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Set the start and end date for the dataset (5 years from today)
start_date = datetime.now()
end_date = start_date + timedelta(days=10 * 365)

# Define the hospital, department, and admission count patterns
hospitals = ['St. Mary Hospital', 'General Medical Center', 'City Health Center', 'Hope Medical Center']

departments = ['ICU', 'Emergency Department', 'Operating Rooms', 'Respiratory Therapy Units', 'NICU']


HW = [5,10,15,20,25]

HospitalWeight = {i:j for i,j in zip(hospitals,HW)}


# Generate the dataset
dataset = []
current_date = start_date


while current_date <= end_date:
    for hospital in hospitals:
        for department in departments:
            day_of_week = current_date.weekday()  # 0 = Monday, 6 = Sunday
            day_of_month = current_date.day
            
            if department == "ICU":
                
                # Apply pattern based on day of the week and day of the month
                ventilator_count = (100 + 10 * day_of_week - day_of_month) + HospitalWeight[hospital]
                ventilator_count = max(0, ventilator_count)  # Ensure non-negative values
                
                
            if department == "Emergency Department":
                
                # Apply pattern based on day of the week and day of the month
                ventilator_count = (80 + 10 * day_of_week - (day_of_month)/2) + HospitalWeight[hospital]
                ventilator_count = max(0, ventilator_count)  # Ensure non-negative values
                
            if department == "Operating Rooms":
                
                # Apply pattern based on day of the week and day of the month
                ventilator_count = (60 + 10 * day_of_week - (day_of_month)/3)+ HospitalWeight[hospital]
                ventilator_count = max(0, ventilator_count)  # Ensure non-negative values
                
            if department == "Respiratory Therapy Units":
                
                # Apply pattern based on day of the week and day of the month
                ventilator_count = (40 + 10 * day_of_week - (day_of_month)/4) + HospitalWeight[hospital]
                ventilator_count = max(0, ventilator_count)  # Ensure non-negative values
                
            if department == "NICU":
                
                # Apply pattern based on day of the week and day of the month
                ventilator_count = (20 + 10 * day_of_week - (day_of_month/5)) + HospitalWeight[hospital]
                ventilator_count = max(0, ventilator_count)  # Ensure non-negative values
                
            
            dataset.append({
                'Date': current_date,
                'Hospital': hospital,
                'Department': department,
                'VentilatorCount': int(ventilator_count),
            })

    current_date += timedelta(days=1)

# Create pandas DataFrame from the dataset
df = pd.DataFrame(dataset)


# Save the dataset as CSV
filename = r'..\data\ventilator_demand.csv'
df.to_csv(filename, index=False)
print(f"Dataset stored as {filename}")


Dataset stored as ..\data\ventilator_demand.csv


In [1]:
import pandas as pd

data = pd.read_csv(r"..\data\ventilator_demand.csv")

In [2]:
data.columns

Index(['Date', 'Hospital', 'Department', 'VentilatorCount'], dtype='object')

In [22]:
import pandas as pd

def create_date_features(dataframe, date_column):
    # Convert the date column to datetime type
    dataframe[date_column] = pd.to_datetime(dataframe[date_column],dayfirst=True)
    
    dataframe['day_of_week'] = dataframe[date_column].dt.weekday
    dataframe['Day'] = dataframe[date_column].dt.day
    
    return dataframe

data = create_date_features(data,"Date")


In [23]:
data.columns

Index(['Date', 'Hospital', 'Department', 'VentilatorCount', 'day_of_week',
       'Day'],
      dtype='object')

In [24]:
cat_coumns = ["Hospital","Department"]

num_column =["day_of_week","Day","VentilatorCount"]

dummies = pd.get_dummies(data[cat_coumns],drop_first=True)

data_new = pd.concat([dummies,data[num_column]],axis=1)

In [25]:
data_new.columns

Index(['Hospital_General Medical Center', 'Hospital_Hope Medical Center',
       'Hospital_St. Mary Hospital', 'Department_ICU', 'Department_NICU',
       'Department_Operating Rooms', 'Department_Respiratory Therapy Units',
       'day_of_week', 'Day', 'VentilatorCount'],
      dtype='object')

In [62]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Splitting data into train test split

target,features = data_new["VentilatorCount"],data_new.drop("VentilatorCount",axis=1)

features = features.astype("int")

x_train, x_test, y_train, y_test = train_test_split(features, target)

model = XGBRegressor()

model.fit(x_train,y_train)

### Evaluating the model

from sklearn.metrics import mean_absolute_error, r2_score,mean_squared_error

predicted = model.predict(x_test)

print("R2 Score is : ",r2_score(predicted,y_test))

print("Mean Squared Error is : ",mean_squared_error(predicted,y_test))

print("Mean Absolute Error is : ",mean_absolute_error(predicted,y_test))

model.save_model(r"..\models\ventilator_model.model")

R2 Score is :  0.9999474310396553
Mean Squared Error is :  0.05609753258716827
Mean Absolute Error is :  0.1693974687445167


In [40]:
model.predict(x_train.iloc[15:16])[0]

96.74022

In [28]:
y_train.iloc[15:16:]


71820    96
Name: VentilatorCount, dtype: int64

In [56]:
def get_ventilator_sample():

    ventilator_sample = {'Hospital_General Medical Center': [0],
    'Hospital_Hope Medical Center': [0],
    'Hospital_St. Mary Hospital': [0],
    'Department_ICU': [0],
    'Department_NICU': [0],
    'Department_Operating Rooms': [0],
    'Department_Respiratory Therapy Units': [0],
    'day_of_week': [0],
    'Day': [0]}

    return ventilator_sample

In [57]:
def prepare_ventilator_sample(current_date,department):
    
    sample_admission = get_ventilator_sample()
    
    sample_admission['day_of_week'] = current_date.weekday() 
    sample_admission['Day'] = current_date.day
    if department =="Department_Emergency Department":
        
        pass
        
    else:
            
        sample_admission[department] = [1]

        
    return pd.DataFrame(sample_admission)


In [58]:
def get_next_fewdays_ventilator_demand():

    ventilator_output ={

        "Next day":{
        "Department_ICU":0,
        "Department_NICU":0,
        "Department_Operating Rooms":0,
        "Department_Respiratory Therapy Units":0,
                    },
        "Next 2 days":{
        "Department_ICU":0,
        "Department_NICU":0,
        "Department_Operating Rooms":0,
        "Department_Respiratory Therapy Units":0,
                    },
        "Next 3 days":{
        "Department_ICU":0,
        "Department_NICU":0,
        "Department_Operating Rooms":0,
        "Department_Respiratory Therapy Units":0,
                    }
    }


    from datetime import datetime

    from datetime import timedelta

    today = datetime.today()

    tomorrow = today+timedelta(days=1)

    day_after_tomorrow = today+timedelta(days=2)

    next_to_DFT = today+timedelta(days=3)

    departments = ['Department_ICU', 'Department_NICU', 'Department_Operating Rooms', 'Department_Respiratory Therapy Units','Department_Emergency Department']

    next_few_days = [tomorrow,day_after_tomorrow,next_to_DFT]


    for ind,date in enumerate(next_few_days):

        for department in departments:

            admission_input = prepare_ventilator_sample(date,department)

            ventolator_count = int(model.predict(admission_input)[0])

            if department in ventilator_output[list(ventilator_output.keys())[ind]].keys():

                ventilator_output[list(ventilator_output.keys())[ind]][department] = ventolator_count

            else:

                ventilator_output[list(ventilator_output.keys())[ind]]['Department_Emergency Department'] = ventolator_count
                    
    return ventilator_output


In [59]:
out = get_next_fewdays_ventilator_demand()

In [60]:
def get_total_ventilator(admission_output):
    
    output = {}
    
    for day in admission_output.keys():
        
        total = 0
        
        for department in admission_output[day].keys():
            
            total+=admission_output[day][department]
            
        output[day] = total
        
    return output

In [61]:
get_total_ventilator(out) 

{'Next day': 395, 'Next 2 days': 444, 'Next 3 days': 491}